In [2]:
import sklearn
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('titanic/train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### easy  
##### для данных с Титаника обучить модели логистической регрессии и решающего дерева на 80% обучающей выборки и проверить результат на 20%
##### у какой модели результат лучше?

отбор нужных данных, замена текстовых данных на числовые

In [4]:
df.loc[df['Sex']=='female', 'Sex'] = 1
df.loc[df['Sex']=='male', 'Sex'] = 0
df_ml = df[['Survived','Pclass','Sex','Age']].fillna(0)
df_ml

,Survived,Pclass,Sex,Age
0,0,3,0,22.0
1,1,1,1,38.0
2,1,3,1,26.0
3,1,1,1,35.0
4,0,3,0,35.0
...,...,...,...,...
886,0,2,0,27.0
887,1,1,1,19.0
888,0,3,1,0.0
889,1,1,0,26.0


определение x и y

In [5]:
x = df_ml[['Pclass','Sex','Age']]
x.head()

,Pclass,Sex,Age
0,3,0,22.0
1,1,1,38.0
2,3,1,26.0
3,1,1,35.0
4,3,0,35.0


In [6]:
y = df_ml[['Survived']]
y.head()

,Survived
0,0
1,1
2,1
3,1
4,0


разбивка на train и test (80% - 20%)

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import sklearn
from sklearn.metrics import accuracy_score

In [52]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8)

**обучение модели логистической регрессии**

In [53]:
model_1 = LogisticRegression(max_iter=1000)
model_1.fit(x_train, y_train)
y_pred = model_1.predict(x_test)
a1 = accuracy_score(y_pred, y_test)
a1

C:\Users\79216\anaconda3\envs\py38\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.770949720670391

**обучение модели решающего дерева**

In [54]:
from sklearn.tree import DecisionTreeClassifier

In [55]:
model_2 = DecisionTreeClassifier()
model_2.fit(x_train, y_train)
y_pred = model_2.predict(x_test)
a2 = accuracy_score(y_pred, y_test)
a2

0.7821229050279329

**сравнение результатов**

In [56]:
print('модель логистической регрессии - ',round(a1*100,2),'%')
print('модель решающего дерева - ',round(a2*100,2),'%')

модель логистической регрессии -  77.09 %
модель решающего дерева -  78.21 %


при нормализации данных точность значительно падает:

In [57]:
from sklearn import preprocessing

In [58]:
x_n=preprocessing.normalize(x)

In [59]:
x_train_n, x_test_n, y_train_n, y_test_n = train_test_split(x_n, y, train_size=0.8)

In [60]:
model_1n = LogisticRegression(max_iter=1000)
model_1n.fit(x_train_n, y_train_n)
y_pred_n = model_1n.predict(x_test_n)
accuracy_score(y_pred_n, y_test_n)

C:\Users\79216\anaconda3\envs\py38\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.6368715083798883

In [61]:
model_2n = DecisionTreeClassifier()
model_2n.fit(x_train, y_train)
y_pred = model_2n.predict(x_test)
accuracy_score(y_pred, y_test)

0.7821229050279329

### normal  
##### используя кросс-валидацию, применить к данным Титаника модель RandomForestClassifier; сравнить с первыми двумя моделями

**обучение модели RandomForestClassifier**

In [62]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [63]:
model_3 = RandomForestClassifier()
model_3.fit(x_train, y_train)
y_pred = model_3.predict(x_test)
a3 = accuracy_score(y_pred, y_test)

C:\Users\79216\AppData\Local\Temp/ipykernel_14568/359069533.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_3.fit(x_train, y_train)


In [64]:
cr_val = cross_val_score(model_3, x_train, y_train)
a4 = accuracy_score(model_3.predict(x_test), y_test)
print('кросс-валидация на train выборке:',cr_val)
print('cреднее по кросс-валидации:', cr_val.mean())
print('качество на тестовой выборке:', accuracy_score(model_3.predict(x_test), y_test))

C:\Users\79216\anaconda3\envs\py38\lib\site-packages\sklearn\model_selection\_validation.py:681: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\79216\anaconda3\envs\py38\lib\site-packages\sklearn\model_selection\_validation.py:681: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\79216\anaconda3\envs\py38\lib\site-packages\sklearn\model_selection\_validation.py:681: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\79216\anaconda3\envs\py38\lib\site-packages\sklearn\model_selection\_validation.py:681: Dat

кросс-валидация на train выборке: [0.79020979 0.78321678 0.77464789 0.83098592 0.83098592]
cреднее по кросс-валидации: 0.8020092583472864
качество на тестовой выборке: 0.776536312849162


**сравнение результатов**

In [65]:
cr_val = cross_val_score(model_1, x_train, y_train)
a5 = accuracy_score(model_1.predict(x_test), y_test)

cr_val = cross_val_score(model_2, x_train, y_train)
a6 = accuracy_score(model_2.predict(x_test), y_test)

C:\Users\79216\anaconda3\envs\py38\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\79216\anaconda3\envs\py38\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\79216\anaconda3\envs\py38\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\79216\anaconda3\envs\py38\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please 

In [67]:
print('модель логистической регрессии - ',round(a1*100,2),'%')
print('модель решающего дерева - ',round(a2*100,2),'%')
print('модель RandomForestClassifier - ',round(a3*100,2),'%')
print('модель логистической регрессии с кросс-валидацией - ',round(a5*100,2),'%')
print('модель решающего дерева с кросс-валидацией - ',round(a6*100,2),'%')
print('модель RandomForestClassifier с кросс-валидацией - ',round(a4*100,2),'%')

модель логистической регрессии -  77.09 %
модель решающего дерева -  78.21 %
модель RandomForestClassifier -  77.65 %
модель логистической регрессии с кросс-валидацией -  77.09 %
модель решающего дерева с кросс-валидацией -  78.21 %
модель RandomForestClassifier с кросс-валидацией -  77.65 %
